# Parameterized Circuits - Code Laboratory

**Section 3: Create Circuits - Part 3** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

### Parameter Creation
| Class | Signature | Returns | Use When |
|-------|-----------|---------|----------|
| `Parameter` | `Parameter(name)` | `Parameter` | Single symbolic parameter |
| `ParameterVector` | `ParameterVector(name, length)` | `ParameterVector` | Multiple parameters efficiently |

### Parameter Binding (⚠️ EXAM CRITICAL!)
| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `qc.assign_parameters()` | `assign_parameters(parameters, inplace=False)` | `QuantumCircuit` | Bind values (returns NEW circuit!) |
| `qc.parameters` | property | `set` | Get unbound parameters |
| `qc.num_parameters` | property | `int` | Count of parameters |

### ParameterExpression Methods
| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `expr.gradient()` | `gradient(param)` | `ParameterExpression` | VQE gradients |
| `expr.subs()` | `subs({old: new})` | `ParameterExpression` | Parameter substitution |
| `expr.bind()` | `bind({param: value})` | `ParameterExpression` | Bind to numeric |
| `expr.numeric()` | `numeric()` | `float` | Get numeric value (after full bind) |

### ⚠️ CRITICAL TRAP
```python
# ❌ WRONG - Original unchanged!
qc.assign_parameters({theta: 0.5})

# ✅ CORRECT - Capture the return!
bound_qc = qc.assign_parameters({theta: 0.5})
```

---

In [ ]:
"""
Qiskit Code Laboratory - Parameterized Circuits
================================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import Statevector, Operator

# Utility functions for this notebook
def verify_state(qc, expected_state, tolerance=1e-10):
    """Verify circuit produces expected state."""
    actual = Statevector.from_instruction(qc)
    return np.allclose(actual.data, expected_state, atol=tolerance)

def show_state(qc, label=""):
    """Display statevector with optional label."""
    sv = Statevector.from_instruction(qc)
    print(f"{label}: {sv}" if label else f"{sv}")
    return sv

def count_params(qc):
    """Count and display parameters in circuit."""
    params = qc.parameters
    print(f"Parameters: {params}")
    print(f"Count: {len(params)}")
    return len(params)

print("✅ Environment ready - Parameterized Circuits Lab")

---

## `Parameter` - Single Symbolic Parameter

### Signature
```python
Parameter(name: str, symbol: Optional[Symbol] = None) -> Parameter
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `name` | `str` | Yes | - | Name for the parameter |

### Returns
`Parameter` - Symbolic parameter for use in circuits

---

### Basic Usage

In [ ]:
# ============================================================
# Parameter - BASIC USAGE
# ============================================================

# Create parameter
theta = Parameter('θ')
print(f"Parameter: {theta}")
print(f"Name: {theta.name}")
print(f"Type: {type(theta)}")

# Use in circuit
qc = QuantumCircuit(1)
qc.ry(theta, 0)

print(f"\nCircuit with parameter:")
print(qc.draw())

# Verify properties
assert len(qc.parameters) == 1, "Should have 1 parameter"
assert qc.num_parameters == 1, "num_parameters should be 1"
print(f"\nParameters: {qc.parameters}")
print(f"Num parameters: {qc.num_parameters}")
print("\n✅ Circuit is symbolic (not executable yet!)")

---

## `assign_parameters()` - Binding Values (⚠️ MOST TESTED!)

### Signature
```python
QuantumCircuit.assign_parameters(parameters, inplace=False, flat_input=False) -> QuantumCircuit
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `parameters` | `dict` | Yes | - | `{param: value}` or `{vector: list}` |
| `inplace` | `bool` | No | `False` | Modify in place (not recommended) |

### Returns
`QuantumCircuit` - NEW circuit with bound parameters

---

### Basic Usage

In [ ]:
# ============================================================
# assign_parameters() - BASIC USAGE
# ============================================================

theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.ry(theta, 0)

# Bind parameter to value
bound_circuit = qc.assign_parameters({theta: np.pi/4})

print("Original circuit:")
print(qc.draw())
print(f"Parameters: {qc.parameters}")

print("\nBound circuit (θ = π/4):")
print(bound_circuit.draw())
print(f"Parameters: {bound_circuit.parameters}")

# Verify: original unchanged, bound has no params
assert len(qc.parameters) == 1, "Original should still have parameter"
assert len(bound_circuit.parameters) == 0, "Bound should have no parameters"
print("\n✅ Original unchanged, bound circuit is executable!")

In [ ]:
# ============================================================
# assign_parameters() - VQE PATTERN: Multiple Bindings
# ============================================================

theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.ry(theta, 0)

# Bind to multiple values (VQE optimization loop pattern)
angles = [0, np.pi/4, np.pi/2, np.pi]

print("VQE Pattern: Same circuit, different parameter values")
print("="*50)

for angle in angles:
    bound = qc.assign_parameters({theta: angle})
    sv = Statevector.from_instruction(bound)
    print(f"\nθ = {angle:.4f}:")
    print(f"  State: {sv}")

print("\n✅ EXAM PATTERN: VQE loop binds different values each iteration!")

---

### ⚠️ TRAP DEMONSTRATION: assign_parameters() Returns NEW Circuit!

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: assign_parameters() Returns NEW Circuit!
# This is the #1 tested trap for parameterized circuits!
# ============================================================

print("⚠️ TRAP: assign_parameters() Returns NEW Circuit!")
print("="*60)

theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.ry(theta, 0)

print("Before binding:")
print(f"  qc.parameters = {qc.parameters}")

# ❌ WRONG - Not capturing return value!
print("\n❌ WRONG approach:")
qc.assign_parameters({theta: 0.5})  # Return value IGNORED!
print(f"  After qc.assign_parameters(): qc.parameters = {qc.parameters}")
print("  ↑ Original circuit UNCHANGED!")

# ✅ CORRECT - Capture the return
print("\n✅ CORRECT approach:")
bound_qc = qc.assign_parameters({theta: 0.5})  # Capture return!
print(f"  bound_qc = qc.assign_parameters({...})")
print(f"  bound_qc.parameters = {bound_qc.parameters}")
print("  ↑ New circuit has no parameters!")

print("\n" + "="*60)
print("💡 RULE: Always capture: bound = qc.assign_parameters(...)")
print("="*60)

---

## `ParameterVector` - Multiple Parameters

### Signature
```python
ParameterVector(name: str, length: int) -> ParameterVector
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `name` | `str` | Yes | - | Base name for parameters |
| `length` | `int` | Yes | - | Number of parameters |

### Access
- `params[i]` - Access i-th parameter
- `len(params)` - Get length
- Iterate: `for p in params`

---

### Basic Usage

In [ ]:
# ============================================================
# ParameterVector - BASIC USAGE
# ============================================================

# Create parameter vector
params = ParameterVector('θ', 3)

print(f"ParameterVector: {params}")
print(f"Length: {len(params)}")
print(f"Individual parameters: {params[0]}, {params[1]}, {params[2]}")

# Use in circuit
qc = QuantumCircuit(3)
for i in range(3):
    qc.ry(params[i], i)

print(f"\nCircuit with parameter vector:")
print(qc.draw())

# Verify
assert qc.num_parameters == 3, "Should have 3 parameters"
print(f"\nParameters: {qc.parameters}")
print("✅ ParameterVector more efficient than multiple Parameter objects!")

In [ ]:
# ============================================================
# ParameterVector - BINDING WITH LIST
# ============================================================

params = ParameterVector('θ', 3)
qc = QuantumCircuit(3)
for i in range(3):
    qc.ry(params[i], i)

# Bind vector with list (order matters!)
values = [np.pi/4, np.pi/3, np.pi/2]
bound_qc = qc.assign_parameters({params: values})

print("Bind with list [π/4, π/3, π/2]:")
print(bound_qc.draw())

# Verify all bound
assert len(bound_qc.parameters) == 0, "All parameters should be bound"
print(f"\nParameters remaining: {len(bound_qc.parameters)}")
print("✅ List binding: values[i] → params[i]")

---

## VQE Ansatz Pattern (⚠️ GUARANTEED EXAM QUESTION!)

### The Hardware-Efficient Ansatz

**Structure**: Rotation Layer → Entangling Layer → Repeat

---

In [ ]:
# ============================================================
# VQE ANSATZ PATTERN - EXAM STRUCTURE
# ============================================================

def create_vqe_ansatz(n_qubits, depth):
    """
    Hardware-efficient ansatz for VQE.
    
    EXAM PATTERN - MEMORIZE THIS STRUCTURE:
    1. Create ParameterVector
    2. Add rotation layers (RY, RZ)
    3. Add entangling layer (CX chain)
    4. Repeat for depth
    """
    qc = QuantumCircuit(n_qubits)
    
    # Parameters: 2 angles per qubit per layer
    params = ParameterVector('θ', n_qubits * depth * 2)
    idx = 0
    
    for d in range(depth):
        # Rotation layer (Y rotations)
        for i in range(n_qubits):
            qc.ry(params[idx], i)
            idx += 1
        
        # Rotation layer (Z rotations)
        for i in range(n_qubits):
            qc.rz(params[idx], i)
            idx += 1
        
        # Entangling layer (CX chain)
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)
    
    return qc, params

# Create 3-qubit, depth-2 ansatz
ansatz, params = create_vqe_ansatz(3, 2)

print("VQE Ansatz (3 qubits, depth 2):")
print(ansatz.draw())

# Verify parameter count
expected_params = 3 * 2 * 2  # n_qubits * depth * 2
assert len(params) == expected_params, f"Expected {expected_params} params"
print(f"\n✅ Total parameters: {len(params)} = 3 qubits × 2 depths × 2 angles")

In [ ]:
# ============================================================
# VQE OPTIMIZATION LOOP - EXAM PATTERN
# ============================================================

print("VQE Optimization Loop Pattern")
print("="*50)

# Initial random parameters
initial_params = np.random.random(len(params)) * 2 * np.pi

print("\n# VQE Loop Structure (MEMORIZE THIS!):")
print("""
for iteration in range(max_iterations):
    # 1. Bind current parameters
    bound_circuit = ansatz.assign_parameters({params: current_params})
    
    # 2. Run on quantum computer (Estimator)
    job = estimator.run([bound_circuit], [hamiltonian])
    energy = job.result().values[0]
    
    # 3. Optimize parameters (classical optimizer)
    current_params = optimizer.step(energy)
    
    # 4. Check convergence
    if converged:
        break
""")

# Demonstrate binding in loop
print("Example: Binding ansatz for one iteration:")
bound_ansatz = ansatz.assign_parameters({params: initial_params})
print(f"  Parameters before: {ansatz.num_parameters}")
print(f"  Parameters after:  {bound_ansatz.num_parameters}")
print("\n✅ This pattern appears on EVERY VQE exam question!")

---

## Parameter Expressions - Arithmetic with Parameters

Parameters support arithmetic operations creating `ParameterExpression` objects.

---

In [ ]:
# ============================================================
# PARAMETER EXPRESSIONS - Arithmetic
# ============================================================

theta = Parameter('θ')
phi = Parameter('φ')

qc = QuantumCircuit(1)
qc.rx(theta, 0)
qc.ry(2*theta, 0)        # Expression: 2θ
qc.rz(theta + phi, 0)    # Expression: θ + φ

print("Circuit with parameter expressions:")
print(qc.draw())
print(f"\nParameters: {qc.parameters}")

# Bind all
bound = qc.assign_parameters({theta: np.pi/4, phi: np.pi/6})
print("\nBound circuit (θ=π/4, φ=π/6):")
print(bound.draw())

# Verify
assert len(bound.parameters) == 0, "All should be bound"
print("✅ Expressions bound correctly")

---

## Partial Binding - Bind Some Parameters

---

In [ ]:
# ============================================================
# PARTIAL BINDING - Bind Some Parameters
# ============================================================

theta = Parameter('θ')
phi = Parameter('φ')

qc = QuantumCircuit(1)
qc.rx(theta, 0)
qc.ry(phi, 0)

print("Original circuit:")
print(qc.draw())
print(f"Parameters: {qc.parameters}")

# Bind only theta
partial = qc.assign_parameters({theta: np.pi/4})

print("\nAfter binding θ only:")
print(partial.draw())
print(f"Parameters remaining: {partial.parameters}")

# Verify partial binding
assert len(partial.parameters) == 1, "Should have 1 param left"
assert phi in partial.parameters, "phi should remain"
print("\n✅ Partial binding: bind incrementally!")

---

## ParameterExpression Methods (Advanced)

### Key Methods
| Method | Purpose |
|--------|---------|
| `gradient(param)` | Symbolic derivative |
| `subs({old: new})` | Parameter substitution |
| `bind({param: val})` | Bind to numeric |
| `numeric()` | Get numeric value |

---

In [ ]:
# ============================================================
# ParameterExpression - Creating Expressions
# ============================================================

theta = Parameter('θ')
phi = Parameter('φ')

# Arithmetic creates ParameterExpressions
expr1 = 2 * theta              # Scalar multiplication
expr2 = theta + phi            # Addition
expr3 = theta ** 2             # Exponentiation
expr4 = theta / np.pi          # Division

print("ParameterExpression Examples:")
print(f"  2 * θ = {expr1}")
print(f"  θ + φ = {expr2}")
print(f"  θ² = {expr3}")
print(f"  θ/π = {expr4}")

print(f"\nType of 2*theta: {type(expr1)}")
print(f"Parameters in θ + φ: {expr2.parameters}")

# Verify type
from qiskit.circuit import ParameterExpression
assert isinstance(expr1, ParameterExpression), "Should be ParameterExpression"
print("✅ Arithmetic creates ParameterExpression")

---

### gradient() and subs() Methods

In [ ]:
# ============================================================
# gradient() - Symbolic Derivatives
# ============================================================

theta = Parameter('θ')

# Expression: f(θ) = θ² + 2θ
expr = theta ** 2 + 2 * theta

# Compute symbolic derivative: d/dθ = 2θ + 2
gradient = expr.gradient(theta)

print("Gradient Computation:")
print(f"  f(θ) = {expr}")
print(f"  d/dθ f(θ) = {gradient}")

# Evaluate gradient at specific point
bound_grad = gradient.bind({theta: 1.0})
print(f"\n  At θ=1: gradient = {bound_grad.numeric()}")  # 2*1 + 2 = 4

# Verify
assert bound_grad.numeric() == 4.0, "Gradient at θ=1 should be 4"
print("✅ gradient() for VQE parameter-shift rule")

In [ ]:
# ============================================================
# subs() - Parameter Substitution
# ============================================================

theta = Parameter('θ')
phi = Parameter('φ')

# Original expression
expr = 2 * theta + 1
print(f"Original: {expr}")

# Substitute theta with phi (parameter → parameter)
new_expr = expr.subs({theta: phi})
print(f"Substitute θ→φ: {new_expr}")

# Substitute with numeric value
numeric_expr = expr.subs({theta: 0.5})
print(f"Substitute θ→0.5: {numeric_expr}")
print(f"  Numeric value: {numeric_expr.numeric():.4f}")

# Verify
assert numeric_expr.numeric() == 2.0, "2*0.5 + 1 = 2"
print("✅ subs() for parameter-shift rule")

---

## 🏋️ Code Challenges

> These are CODING challenges - solve them by writing code.
> For conceptual questions, see [README.md](./README.md) practice section.

---

In [ ]:
# ============================================================
# CHALLENGE 1: VQE Ansatz
# ============================================================
# Task: Create a 2-qubit, depth-1 ansatz with 4 parameters
# Structure: RY on each qubit, then CX(0,1)
# ============================================================

def challenge_1():
    """
    Create a 2-qubit ansatz with:
    - 4 parameters (2 qubits × 2 rotations)
    - RY(θ0), RY(θ1) on qubits 0, 1
    - RZ(θ2), RZ(θ3) on qubits 0, 1  
    - CX(0, 1) entangling
    
    Returns:
        tuple: (QuantumCircuit, ParameterVector)
    """
    # YOUR CODE HERE
    params = ParameterVector('θ', 4)
    qc = QuantumCircuit(2)
    # TODO: Add rotations and entangling
    
    return qc, params

# Uncomment to test:
# qc, params = challenge_1()
# print("Your ansatz:")
# print(qc.draw())
# if qc.num_parameters == 4 and qc.num_qubits == 2:
#     print("✅ Challenge 1 PASSED!")
# else:
#     print("❌ Challenge 1 FAILED")

In [ ]:
# ============================================================
# SOLUTION: Challenge 1
# ============================================================

def challenge_1_solution():
    """Reference solution for Challenge 1."""
    params = ParameterVector('θ', 4)
    qc = QuantumCircuit(2)
    
    # RY rotations
    qc.ry(params[0], 0)
    qc.ry(params[1], 1)
    
    # RZ rotations
    qc.rz(params[2], 0)
    qc.rz(params[3], 1)
    
    # Entangling
    qc.cx(0, 1)
    
    return qc, params

qc_sol, params_sol = challenge_1_solution()
print("Solution ansatz:")
print(qc_sol.draw())
print(f"\n✅ {qc_sol.num_parameters} parameters, {qc_sol.num_qubits} qubits")

In [ ]:
# ============================================================
# CHALLENGE 2: assign_parameters() Trap Detection
# ============================================================
# Task: Identify and fix the CRITICAL exam trap in this code
# This trap appears in ~15% of exam questions!
# ============================================================

def challenge_2():
    """
    BUGGY CODE - Find and fix the bug!
    
    This code is supposed to create TWO different circuits:
    - One with theta = π/4
    - One with theta = π/2
    
    But it doesn't work! Why?
    """
    theta = Parameter('θ')
    qc = QuantumCircuit(1)
    qc.ry(theta, 0)
    
    # Bind to two different values
    qc.assign_parameters({theta: np.pi/4})  # First binding
    qc.assign_parameters({theta: np.pi/2})  # Second binding
    
    # Check if theta is still unbound
    print(f"Parameters still unbound: {qc.parameters}")
    # What went wrong?
    
# Run the buggy code to see the trap:
# challenge_2()

# TODO: Fix the code to correctly create two bound circuits

In [ ]:
# ============================================================
# SOLUTION: Challenge 2 - assign_parameters() TRAP
# ============================================================

def challenge_2_solution():
    """
    The TRAP: assign_parameters() returns a NEW circuit!
    It does NOT modify the original circuit in-place!
    
    This is the #1 most common parameterized circuit mistake.
    """
    theta = Parameter('θ')
    qc = QuantumCircuit(1)
    qc.ry(theta, 0)
    
    # ❌ WRONG - Original qc is unchanged!
    print("❌ WRONG way (original unchanged):")
    qc.assign_parameters({theta: np.pi/4})  # Returns new circuit, but we ignore it!
    print(f"   qc still has parameters: {qc.parameters}")
    
    # ✅ CORRECT - Capture the return value!
    print("\n✅ CORRECT way (capture return):")
    bound_qc_1 = qc.assign_parameters({theta: np.pi/4})
    bound_qc_2 = qc.assign_parameters({theta: np.pi/2})
    
    print(f"   bound_qc_1 parameters: {bound_qc_1.parameters}")  # Empty set
    print(f"   bound_qc_2 parameters: {bound_qc_2.parameters}")  # Empty set
    
    # Verify both are bound
    assert len(bound_qc_1.parameters) == 0, "Should be fully bound"
    assert len(bound_qc_2.parameters) == 0, "Should be fully bound"
    
    print("\n✅ Challenge 2 PASSED: assign_parameters() trap understood!")
    return bound_qc_1, bound_qc_2

# Run the solution
bound1, bound2 = challenge_2_solution()
print(f"\nCircuit 1 (θ=π/4): {bound1.data[0].operation.params}")
print(f"Circuit 2 (θ=π/2): {bound2.data[0].operation.params}")

In [ ]:
# ============================================================
# CHALLENGE 3: Partial Binding
# ============================================================
# Task: Understand how partial binding works
# Exam question: "What is len(qc.parameters) after partial binding?"
# ============================================================

def challenge_3():
    """
    Given a circuit with 3 parameters (theta, phi, gamma),
    what happens if you only bind 2 of them?
    
    Task: Create a circuit, do partial binding, check remaining parameters
    """
    theta = Parameter('θ')
    phi = Parameter('φ')
    gamma = Parameter('γ')
    
    qc = QuantumCircuit(1)
    qc.rx(theta, 0)
    qc.ry(phi, 0)
    qc.rz(gamma, 0)
    
    print(f"Before binding: {len(qc.parameters)} parameters")
    print(f"  Parameters: {qc.parameters}")
    
    # Partial binding - only theta and phi
    partial = qc.assign_parameters({theta: 0.5, phi: 1.0})
    
    print(f"\nAfter partial binding: {len(partial.parameters)} parameters")
    print(f"  Remaining: {partial.parameters}")
    
    # Verify
    assert len(partial.parameters) == 1, "Should have 1 unbound"
    assert gamma in partial.parameters, "gamma should still be unbound"
    
    print("\n✅ Challenge 3 PASSED: Partial binding understood!")
    return partial

# Run
partial_circuit = challenge_3()

In [ ]:
# ============================================================
# CHALLENGE 3: Same Name ≠ Same Parameter Trap
# ============================================================
# Task: Predict what happens when two Parameters have the same name
# ============================================================

def challenge_3():
    """
    EXAM QUESTION: How many parameters does this circuit have?
    
    Answer before running, then verify!
    """
    theta1 = Parameter('θ')
    theta2 = Parameter('θ')  # Same name as theta1!
    
    qc = QuantumCircuit(2)
    qc.ry(theta1, 0)
    qc.ry(theta2, 1)
    
    # Question: How many parameters? 1 or 2?
    # Your answer: _____
    
    return qc, theta1, theta2

# Uncomment to reveal the answer:
# qc, t1, t2 = challenge_3()
# print(f"Number of parameters: {qc.num_parameters}")
# print(f"Are they the same object? {t1 is t2}")

---

## 📚 Resources

### Code References
- [Qiskit Parameter API](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.Parameter)
- [Qiskit ParameterVector API](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.ParameterVector)

### Related Materials
- [README.md](./README.md) - Conceptual explanations, traps, mnemonics
- [Practice Questions](./README.md#practice-exam) - Text-based exam practice

### Next Notebook
- [Circuit Library](./circuit_library.ipynb) - Pre-built circuits and gates

---

### Quick Reference Card

```
┌─────────────────────────────────────────────────────────────────┐
│ PARAMETERIZED CIRCUITS - QUICK REFERENCE                        │
├─────────────────────────────────────────────────────────────────┤
│ CREATION:                                                       │
│   theta = Parameter('θ')         - Single parameter            │
│   params = ParameterVector('θ', n) - Multiple parameters       │
├─────────────────────────────────────────────────────────────────┤
│ BINDING (⚠️ CRITICAL!):                                         │
│   bound = qc.assign_parameters({theta: value})                 │
│   bound = qc.assign_parameters({params: [v1, v2, ...]})        │
│   ↑ MUST capture return! Original unchanged!                   │
├─────────────────────────────────────────────────────────────────┤
│ PROPERTIES:                                                     │
│   qc.parameters         - Set of unbound parameters            │
│   qc.num_parameters     - Count of parameters                  │
├─────────────────────────────────────────────────────────────────┤
│ VQE PATTERN:                                                    │
│   for iteration in range(max_iter):                            │
│       bound = ansatz.assign_parameters({params: values})       │
│       energy = estimator.run([bound], [H]).result()            │
│       values = optimizer.step(energy)                          │
└─────────────────────────────────────────────────────────────────┘
```

---

**Notebook verified with Qiskit 1.x** | Last updated: 2025